## Setup

In [1]:
# Google Colab setup
# Ignore if you're not running on colabx

GDRIVE_PWD = 'DenoisingAutoencoder'

try:
    from google.colab import drive
    import os
    IN_COLAB = True
except:
    IN_COLAB = False
    
if IN_COLAB:
    drive.mount('/content/gdrive', force_remount=True)
    root_dir = "/content/gdrive/My Drive/"
    base_dir = os.path.join(root_dir, GDRIVE_PWD)
    
    if not os.path.exists(base_dir):
        os.makedirs(base_dir)
    os.chdir(base_dir)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [3]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import keras
from keras import backend as K
import numpy as np
import os
import matplotlib.pylab as plt

Using TensorFlow backend.


**Create necessary directories**

In [0]:
P_MODELSAVE = 'saved_models'
P_LOGS = 'logs'
P_IMGSAVE = 'saved_images'

dirs = [P_MODELSAVE, P_LOGS, P_IMGSAVE]

for d in dirs:
    if not os.path.exists(d):
        os.makedirs(d)

**Important parameteres**

In [0]:
dataset_path = 'data/images'
batch_size = 20
epochs = 150
input_shape = (256, 256)
noise_factor = 1

# the path to save the weight of the model
saved_weight = os.path.join(P_MODELSAVE, 'dataweights.{epoch:02d}-{val_acc:.2f}.hdf5')

## DataGenerator

### Add random crop functionality

**Add random crop fucntionality to Keras' ImageDataGenerator by overriding 'load_img' method in its module**

In [0]:
from PIL import Image as pil_image

def random_crop(img, random_crop_size):
    width, height = img.size # PIL format
    dx, dy = random_crop_size
    x = np.random.randint(0, width - dx + 1)
    y = np.random.randint(0, height - dy + 1)
    return img.crop((x, y, x+dx, y+dy))


def load_img_extended(path, grayscale=False, color_mode='rgb', target_size=None,
                      interpolation='nearest'):
    if grayscale is True:
        warnings.warn('grayscale is deprecated. Please use '
                      'color_mode = "grayscale"')
        color_mode = 'grayscale'
    if pil_image is None:
        raise ImportError('Could not import PIL.Image. '
                          'The use of `array_to_img` requires PIL.')
    img = pil_image.open(path)
    if color_mode == 'grayscale':
        if img.mode != 'L':
            img = img.convert('L')
    elif color_mode == 'rgba':
        if img.mode != 'RGBA':
            img = img.convert('RGBA')
    elif color_mode == 'rgb':
        if img.mode != 'RGB':
            img = img.convert('RGB')
    else:
        raise ValueError('color_mode must be "grayscale", "rbg", or "rgba"')
    
    if target_size is not None:
        width_height_tuple = (target_size[1], target_size[0])
        if img.size != width_height_tuple:
            img = random_crop(img, width_height_tuple) # here comes the magic
    return img

In [0]:
# Overriding method
keras.preprocessing.image.image.load_img = load_img_extended

### ImageDataGenerator

**ImageDataGenerator arguments:**

In [0]:
from keras.preprocessing.image import ImageDataGenerator

data_gen_args = dict(
#     featurewise_center=True,
#     featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    brightness_range=[0.5, 1.2],
    shear_range=0.01,
    horizontal_flip=True,
    rescale=1/255,
    fill_mode='reflect',
    data_format='channels_last')

data_flow_args = dict(
    target_size=input_shape,
    batch_size=batch_size,
    class_mode='input') # Since we want to reconstruct the input

**Add gaussian noise to the input after augmentations have taken place.**  
**Inspired by** https://jkjung-avt.github.io/keras-image-cropping/

In [0]:
def noisy_generator(batches):
    for batch_x, batch_y in batches:
        sigma = np.random.exponential(0.15)
        noise = noise_factor * np.random.normal(scale=sigma, size=batch_x.shape)
        batch_noisy = np.clip(batch_x + noise, 0, 1)
        yield (batch_noisy, batch_y)

## Model

In [0]:
import keras.layers as layers
import keras.models as models
from keras.initializers import orthogonal


def Conv2DLayer(x, filters, kernel, strides, padding, block_id, kernel_init=orthogonal()):
    prefix = f'block_{block_id}_'
    x = layers.Conv2D(filters, kernel_size=kernel, strides=strides, padding=padding,
                      kernel_initializer=kernel_init, name=prefix+'conv')(x)
    x = layers.LeakyReLU(name=prefix+'lrelu')(x)
    x = layers.Dropout(0.2, name=prefix+'drop')((x))
    x = layers.BatchNormalization(name=prefix+'conv_bn')(x)
    return x

def Transpose_Conv2D(x, filters, kernel, strides, padding, block_id, kernel_init=orthogonal()):
    prefix = f'block_{block_id}_'
    x = layers.Conv2DTranspose(filters, kernel_size=kernel, strides=strides, padding=padding,
                               kernel_initializer=kernel_init, name=prefix+'de-conv')(x)
    x = layers.LeakyReLU(name=prefix+'lrelu')(x)
    x = layers.Dropout(0.2, name=prefix+'drop')((x))
    x = layers.BatchNormalization(name=prefix+'conv_bn')(x)
    return x



def AutoEncdoer(input_shape):
    inputs = layers.Input(shape=input_shape)
    
    # 256 x 256
    conv1 = Conv2DLayer(inputs, 64, 3, strides=1, padding='same', block_id=1)
    conv2 = Conv2DLayer(conv1, 64, 3, strides=2, padding='same', block_id=2)
    
    # 128 x 128
    conv3 = Conv2DLayer(conv2, 128, 5, strides=2, padding='same', block_id=3)
    
    # 64 x 64
    conv4 = Conv2DLayer(conv3, 128, 3, strides=1, padding='same', block_id=4)
    conv5 = Conv2DLayer(conv4, 256, 5, strides=2, padding='same', block_id=5)
    
    # 32 x 32
    conv6 = Conv2DLayer(conv5, 512, 3, strides=2, padding='same', block_id=6)
    
    # 16 x 16
    deconv1 = Transpose_Conv2D(conv6, 512, 3, strides=2, padding='same', block_id=7)
    
    # 32 x 32
    skip1 = layers.concatenate([deconv1, conv5], name='skip1')
    conv7 = Conv2DLayer(skip1, 256, 3, strides=1, padding='same', block_id=8)
    deconv2 = Transpose_Conv2D(conv7, 128, 3, strides=2, padding='same', block_id=9)
    
    # 64 x 64
    skip2 = layers.concatenate([deconv2, conv3], name='skip2')
    conv8 = Conv2DLayer(skip2, 128, 5, strides=1, padding='same', block_id=10)
    deconv3 = Transpose_Conv2D(conv8, 64, 3, strides=2, padding='same', block_id=11)
    
    # 128 x 128
    skip3 = layers.concatenate([deconv3, conv2], name='skip3')
    conv9 = Conv2DLayer(skip3, 64, 5, strides=1, padding='same', block_id=12)
    deconv4 = Transpose_Conv2D(conv9, 64, 3, strides=2, padding='same', block_id=13)
    
    # 256 x 256
    skip3 = layers.concatenate([deconv4, conv1])
    conv10 = layers.Conv2D(3, 3, strides=1, padding='same', activation='sigmoid',
                       kernel_initializer=orthogonal(), name='final_conv')(skip3)

    
    return models.Model(inputs=inputs, outputs=conv10)

## Train

**Laod data**

In [0]:
train_datagen = ImageDataGenerator(**data_gen_args)
val_datagen = ImageDataGenerator(**data_gen_args)

train_batches = train_datagen.flow_from_directory(
    dataset_path + '/train',
    **data_flow_args)

val_batches = val_datagen.flow_from_directory(
    dataset_path + '/train',
    **data_flow_args)


train_noisy_batches = noisy_generator(train_batches)
val_noisy_batches = noisy_generator(val_batches)

Found 200 images belonging to 1 classes.
Found 200 images belonging to 1 classes.


**Build and compile the model**

In [0]:
# !pip install keras-adabound

In [0]:
from keras.optimizers import SGD, Adam

model = AutoEncdoer((*input_shape, 3))
# model_opt = SGD(lr=0.005, decay=1-0.995, momentum=0.7, nesterov=False)
model_opt = Adam(lr=0.002)

model.compile(optimizer=model_opt, loss='mse', metrics=['accuracy'])

**Some useful callbacks**

In [0]:
modelchk = keras.callbacks.ModelCheckpoint(saved_weight, 
                                      monitor='val_acc', 
                                      verbose=1,
                                      save_best_only=True, 
                                      save_weights_only=False,
                                      mode='auto',
                                      period=2)

tensorboard = keras.callbacks.TensorBoard(log_dir=P_LOGS,
                                          histogram_freq=0,
                                          write_graph=True,
                                          write_images=True)

csv_logger = keras.callbacks.CSVLogger(f'{P_LOGS}/keras_log.csv',
                                       append=True)

In [0]:
model.fit_generator(train_noisy_batches,
                    steps_per_epoch = train_batches.samples // batch_size,
                    epochs=epochs,
                    verbose=1, 
                    validation_data=val_noisy_batches,
                    validation_steps = train_batches.samples // batch_size,
                    callbacks=[modelchk, tensorboard, csv_logger],
                    use_multiprocessing=True)

## Evaluation

In [0]:
model = keras.models.load_model(os.path.join(P_MODELSAVE,
                                            'phase2_weights.01-0.86.hdf5'))

In [0]:
test_datagen = ImageDataGenerator(**data_gen_args)

test_batches = test_datagen.flow_from_directory(
    dataset_path + '/test',
    **data_flow_args)

test_noisy_batches = noisy_generator(test_batches)

Found 200 images belonging to 1 classes.


In [0]:
X, y = next(test_noisy_batches)

In [0]:
score = model.evaluate(X, y, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

20/20 [==============================] - 33s 2s/step
Test loss: 0.000924995809327811
Test accuracy: 0.9209556579589844


In [0]:
decoded_imgs = model.predict(X)

In [0]:
import matplotlib.pyplot as plt

n = 5

plt.figure(figsize=(40, 15))
for i in range(n):
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(X[i])
    ax.axis('off')

    ax = plt.subplot(2, n, i + n + 1)
    plt.imshow(decoded_imgs[i])
    ax.axis('off')

plt.show()

## Tensorboard

In [90]:
!pip install -q tf-nightly-2.0-preview

     |████████████████████████████████| 79.1MB 1.9MB/s 
     |████████████████████████████████| 450kB 46.1MB/s 
     |████████████████████████████████| 3.2MB 50.7MB/s 


In [0]:
%load_ext tensorboard
%tensorboard --logdir {P_TBOARD}